<a href="https://colab.research.google.com/github/ochaudha/sample/blob/main/FNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai langchain requests python-dotenv


In [ ]:
!pip install python-dotenv requests crewai langchain-ollama urllib3

In [ ]:
# In a Colab cell
!apt-get update
!apt-get install -y tmux

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!apt-get install -y curl


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.20).
0 upgraded, 0 newly installed, 0 to remove and 89 not upgraded.


In [ ]:
import os
os.environ['CREWAI_DISABLE_TELEMETRY'] = 'true'
os.environ['EMAIL_TO'] = "omveer.chaudhary@gmail.com"
os.environ['EMAIL_FROM'] = "omveer.chaudhary@gmail.com"
os.environ['EMAIL_USER'] = "omveer.chaudhary@gmail.com"
os.environ['EMAIL_PASSWORD'] = "password"
os.environ['EMAIL_SERVER'] = "smtp://abc.com:587"


import time
import requests
import subprocess
import logging
from typing import List, Dict
from dataclasses import dataclass
from dotenv import load_dotenv
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.exceptions import SSLError
import urllib3
import json

from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_ollama import OllamaLLM as Ollama

# Load environment variables
load_dotenv()
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)



In [ ]:
# In a Colab cell, before initializing the Ollama LLM
!ollama pull llama3.1:8b

In [ ]:
import subprocess
import time
import os

# ... (previous imports and definitions) ...

def start_ollama_tmux():
    """Starts Ollama in a detached tmux session if it's not already running."""
    session_name = "ollama_server"
    ollama_cmd = "ollama serve"

    # Check if a tmux session with the given name exists
    try:
        subprocess.run(["tmux", "has-session", "-t", session_name], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Tmux session '{session_name}' already exists. Assuming Ollama is running.")
    except subprocess.CalledProcessError:
        # Session does not exist, start a new one
        print(f"Tmux session '{session_name}' not found. Starting Ollama server...")
        try:
            subprocess.run(["tmux", "new-session", "-d", "-s", session_name, ollama_cmd], check=True)
            print("Ollama server started in a detached tmux session.")
            # Give Ollama some time to start
            time.sleep(10)  # Adjust as needed
        except subprocess.CalledProcessError as e:
            print(f"Error starting Tmux session for Ollama: {e}")
            raise # Re-raise the exception to stop execution if Ollama can't start

# ... (rest of your code) ...

def main():
    #validate_env()

    # Start Ollama in a tmux session before initializing the LLM
    start_ollama_tmux()

    # Now, initialize the Ollama LLM as before
    llm = Ollama(model="ollama/llama3.1:8b", temperature=0)

    # ... (rest of your main function) ...

if __name__ == "__main__":
    main()

Tmux session 'ollama_server' already exists. Assuming Ollama is running.


In [ ]:
# Logging config
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# HTTP session with retries
session = requests.Session()
retry_strategy = Retry(total=3, backoff_factor=1, status_forcelist=[502, 503, 504])
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
session.mount("http://", adapter)

@dataclass
class APIEndpoint:
    name: str
    url: str
    expected_status: int = 200

# Define static list of endpoints here
STATIC_ENDPOINTS = [
    APIEndpoint(name="\n\nGoogle ", url="https://www.google.com/")
]

@tool
def api_monitor_tool(_: str = "") -> List[Dict]:
    """Checks the health of API endpoints and returns their status."""
    logging.info("🛠️ Running api_monitor_tool...")
    results = []
    for endpoint in STATIC_ENDPOINTS:
        url = endpoint.url.rstrip("/") + "/"
        status = {
            'name': endpoint.name,
            'url': url,
            'status': 'unknown',
            'status_code': None,
            'error': None
        }
        try:
            logging.info(f"🔍 Checking {endpoint.name} at {url}")
            resp = session.get(url, timeout=10, verify=False, proxies={"http": None, "https": None})
            status['status_code'] = resp.status_code
            status['status'] = 'up' if resp.status_code == endpoint.expected_status else 'down'
        except SSLError as ssl_err:
            status['status'] = 'down'
            status['error'] = f"SSL Error: {ssl_err}"
        except Exception as e:
            status['status'] = 'down'
            status['error'] = str(e)
        results.append(status)
    return results

@tool
def email_alert_tool(api_statuses: str) -> str:
    """Sends email alerts with the status of all APIs."""
    subject = "API Health Check Report"
    body = f"Dear Team,\n\nThe health check report for the API endpoints is as follows:\n{api_statuses}\n\nPlease note any endpoints marked as 'down'.\n\nBest regards,\nAPI Health Checker"
    try:
        with open('/tmp/email_report.txt', 'w') as f:
            f.write(f"Subject: {subject}\n\n{body}")

        recipients = [email.strip() for email in os.getenv("EMAIL_TO", "").split(",") if email.strip()]
        if not recipients:
            return "❌ No email recipients configured"

        curl_command = [
            'curl',
            '--url', os.getenv("EMAIL_SERVER", ""),
            '--mail-from', os.getenv("EMAIL_FROM", ""),
            '--upload-file', '/tmp/email_report.txt',
            '--user', f"{os.getenv('EMAIL_USER', '')}:{os.getenv('EMAIL_PASSWORD', '')}",
            '--ssl-reqd'
        ] + sum([['--mail-rcpt', r] for r in recipients], [])

        subprocess.run(curl_command, check=True)
        return f"✅ Email report sent to: {', '.join(recipients)}"
    except subprocess.CalledProcessError as e:
        return f"❌ Failed to send email (curl error): {str(e)}"
    except Exception as e:
        return f"❌ Failed to send email: {str(e)}"

def format_api_statuses(results: List[Dict]) -> str:
    """Formats the API status results into a readable string for the email body."""
    formatted_statuses = []
    for res in results:
        status_line = f"- {res['name']}: {res['status']}"
        if res['status'] == 'up' and res['status_code']:
            status_line += f" ({res['status_code']})"
        if res['error']:
            status_line += f" (Error: {res['error']})"
        formatted_statuses.append(status_line)
    return "\n".join(formatted_statuses)

def validate_env():
    required_vars = ["EMAIL_FROM", "EMAIL_USER", "EMAIL_PASSWORD", "EMAIL_SERVER", "EMAIL_TO"]
    missing = [var for var in required_vars if not os.getenv(var)]
    if missing:
        raise EnvironmentError(f"Missing environment variables: {', '.join(missing)}")



In [ ]:
def main():
    validate_env()

    llm = Ollama(model="ollama/llama3.1:8b", temperature=0)

    monitor_agent = Agent(
        role="API Health Checker",
        goal="Monitor API endpoints for availability and report failures in a concise format.",
        backstory="You're a vigilant backend engineer constantly monitoring API health and summarizing the status.",
        tools=[api_monitor_tool],
        llm=llm,
        memory=False,
        allow_delegation=False,
        verbose=True,
        planner=False
    )

    reporting_agent = Agent(
        role="API Report Sender",
        goal="Generate and send a report containing the status of all APIs in a well-formatted email.",
        backstory="You are responsible for compiling and distributing API health reports to the team.",
        tools=[email_alert_tool],
        llm=llm,
        memory=False,
        allow_delegation=False,
        verbose=True,
        planner=False
    )

    monitoring_task = Task(
        description="Check the health of these endpoints and summarize their status in a bulleted list format, including the HTTP status code in parentheses if the API is up, and any errors if it's down. Do not include any salutations, closings, subject lines, or overall titles. Just the bulleted list of API statuses.",
        agent=monitor_agent,
        expected_output="A formatted string containing a bulleted list of endpoint statuses with status codes (if up) or errors (if down)."
    )

    reporting_task = Task(
        description="Take the summarized API status and send it via email using the email_alert_tool, ensuring the email body starts with 'Dear Team,' and includes a concluding 'Best regards,' from 'API Health Checker'.",
        agent=reporting_agent,
        expected_output="Confirmation that the email report was sent."
    )

    monitoring_crew = Crew(
        agents=[monitor_agent, reporting_agent],
        tasks=[monitoring_task, reporting_task],
        verbose=True,
        process=Process.sequential
    )

    while True:
        try:

            logging.info("🔄 Starting monitoring cycle...")
            results = monitoring_crew.kickoff()
            logging.info(f"🔍 Monitoring Results: {results}")

            if isinstance(results, str):
                report_body = results
                reporting_task.context = [report_body]
                logging.info("📧 Sending API health report...")
                alert_result = monitoring_crew.kickoff(task=reporting_task)
                logging.info(f"📨 Report Result: {alert_result}")
            else:
                logging.error("❌ Unexpected format for API status report.")

            logging.info("⏳ Waiting 5 minutes...")
            time.sleep(5400) #om uncomment

        except KeyboardInterrupt:
            logging.info("🛑 Monitoring stopped manually.")
            break
        except Exception as e:
            logging.error(f"❌ Error: {str(e)}")
            time.sleep(60)

if __name__ == "__main__":
    main()


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: fe85886a-8ab8-42ac-92c8-3b2d8d9f0a52                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1c9c6320-24f6-4d09-aa0c-11cacb34d184
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1c9c6320-24f6-4d09-aa0c-11cacb34d184
       Status: Executing Task...
    └── 🤖 Agent: API Health Checker
            Status: In Progress

# Agent: API Health Checker
## Task: Check the health of these endpoints and summarize their status in a bulleted list format, including the HTTP status code in parentheses if the API is up, and any errors if it's down. Do not include any salutations, closings, subject lines, or overall titles. Just the bulleted list of API statuses.


🤖 Agent: API Health Checker
    Status: In Progress

...

🚀 Crew: crew
└── 📋 Task: 1c9c6320-24f6-4d09-aa0c-11cacb34d184
       Assigned to: API Health Checker
       Status: ❌ Failed
    └── 🤖 Agent: API Health Checker
            Status: In Progress
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 1c9c6320-24f6-4d09-aa0c-11cacb34d184                                                                     │
│  Agent: API Health Checker                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: fe85886a-8ab8-42ac-92c8-3b2d8d9f0a52                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:root:❌ Error: litellm.APIConnectionError: list index out of range
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/main.py", line 2904, in completion
    response = base_llm_http_handler.completion(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 270, in completion
    data = provider_config.transform_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/ollama/completion/transformation.py", line 322, in transform_request
    modified_prompt = ollama_pt(model=model, messages=messages)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/litellm_core_utils/prompt_templates/factory.py", line 229, in ollama_pt
    tool_calls = messages[msg_i].get("tool_calls")
                 ~~~~~~~~^^^^^^^
IndexError: list index

KeyboardInterrupt: 